In [ ]:
!pip install ultralytics

# YOLOv5 and PyTorch for object detection
from ultralytics import YOLO
import torch

# OpenCV for image processing and real-time video capture
import cv2

# TensorFlow for potential neural network integration
import tensorflow as tf

# NumPy for numerical operations
import numpy as np

# Matplotlib for plotting and visualizing data
import matplotlib.pyplot as plt

# Pandas for data manipulation and analysis
import pandas as pd

# Pathlib for file path operations
import shutil
from pathlib import Path
from tqdm.auto import tqdm


# JSON for handling JSON data
import json

# TensorFlow Keras for image preprocessing
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Scikit-learn for class weight computation and data splitting
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split

# TensorFlow Keras callbacks for early stopping during training
from tensorflow.keras.callbacks import EarlyStopping

# TQDM for progress bars
from tqdm.auto import tqdm

# Shutil for file operations
import shutil

# Pillow (PIL) for image processing
from PIL import Image

# Kaggle API for dataset handling and competition submissions
import kagglehub

#Displaying Results
import random
import os

#Imports for live camera and real-time detection
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

In [ ]:
# Download latest version of the dataset
path = kagglehub.dataset_download("klemenko/kitti-dataset")

print("Path to dataset files:", path)

 53%|█████▎    | 12.0G/22.5G [09:24<08:17, 22.7MB/s]


ChunkedEncodingError: ('Connection broken: IncompleteRead(12836669426 bytes read, 11304182579 more expected)', IncompleteRead(12836669426 bytes read, 11304182579 more expected))

In [ ]:
# Specify the base directory for the KITTI dataset
base_dir = Path('/root/.cache/kagglehub/datasets/klemenko/kitti-dataset/versions/1')

# Define the path to the directory containing the training images
data_path = base_dir / 'data_object_image_2' / 'training' / 'image_2'

# Define the path to the directory containing the training images
test_path = base_dir / 'data_object_image_2' / 'testing' / 'image_2'

# Define the path to the directory containing the corresponding labels for the training images
label_path = base_dir / 'data_object_label_2' / 'training' / 'label_2'

# Define a dictionary mapping class names to integer labels
classes = {
    'Car': 0,              # Label for Car
    'Pedestrian': 1,       # Label for Pedestrian
    'Van': 2,              # Label for Van
    'Cyclist': 3,          # Label for Cyclist
    'Truck': 4,            # Label for Truck
    'Misc': 5,             # Label for miscellaneous objects
    'Tram': 6,             # Label for Tram
    'Person_sitting': 7    # Label for a person sitting
}


In [ ]:
# Generate a sorted list of all files in the training images directory
# This will include all image files located in 'data_path'
data = sorted(list(data_path.glob('*')))

# Generate a sorted list of all files in the labels directory
# This will include all label files located in 'label_path'
labels = sorted(list(label_path.glob('*')))


In [ ]:
# Combine the sorted lists of images and labels into pairs
# Each pair will contain a corresponding image file and its label file
pairs = list(zip(data, labels))

# Display the first two pairs to verify the zipping process
pairs[:2]


In [ ]:
# Split the pairs of images and labels into training and testing sets
# `test_size=0.2` specifies that 20% of the pairs will be used for testing
# `shuffle=True` ensures that the pairs are shuffled before splitting
train, test = train_test_split(pairs, test_size=0.2, shuffle=True)

# Print the number of pairs in the training and testing sets to verify the split
len(train), len(test)


In [ ]:
# Define the path for the training split directory and resolve it to an absolute path
train_split_path = Path('train').resolve()

# Create the training split directory if it does not already exist
train_split_path.mkdir(exist_ok=True)

# Define the path for the validation split directory and resolve it to an absolute path
valid_path = Path('valid').resolve()

# Create the validation split directory if it does not already exist
valid_path.mkdir(exist_ok=True)


In [ ]:

def convert_kitti_to_yolo(label_file, img_width, img_height):
    """Converts KITTI label format to YOLO format."""

    # Open the label file and read all lines
    with open(label_file, 'r') as f:
        lines = f.readlines()

    # Initialize an empty list to store YOLO annotations
    yolo_annotations = []

    # Process each line in the label file
    for line in lines:
        # Split the line into individual data components
        data = line.strip().split(' ')
        class_name = data[0]

        # Check if the class name is in the predefined classes
        if class_name in classes:
            class_id = classes[class_name]  # Get the class ID
            x_min = float(data[4])  # Minimum x-coordinate of the bounding box
            y_min = float(data[5])  # Minimum y-coordinate of the bounding box
            x_max = float(data[6])  # Maximum x-coordinate of the bounding box
            y_max = float(data[7])  # Maximum y-coordinate of the bounding box

            # Normalize coordinates to [0, 1]
            x_center = ((x_min + x_max) / 2) / img_width  # X center of the bounding box
            y_center = ((y_min + y_max) / 2) / img_height  # Y center of the bounding box
            width = (x_max - x_min) / img_width  # Width of the bounding box
            height = (y_max - y_min) / img_height  # Height of the bounding box

            # Append the YOLO formatted annotation to the list
            yolo_annotations.append(f"{class_id} {x_center} {y_center} {width} {height}")

    # Return the list of YOLO annotations
    return yolo_annotations


In [ ]:
# Loop through each pair of training images and labels
for t_img, t_lb in tqdm(train):
    # Define the path for copying the image to the training split directory
    im_path = train_split_path / t_img.name

    # Define the path for copying the label to the training split directory
    lb_path = train_split_path / t_lb.name

    # Copy the training image to the designated path
    shutil.copy(t_img, im_path)

    # Copy the training label to the designated path
    shutil.copy(t_lb, lb_path)

    # Open the copied image to get its dimensions
    img = Image.open(im_path)
    img_width, img_height = img.size

    # Convert the KITTI format label to YOLO format
    yolo_annotations = convert_kitti_to_yolo(t_lb, img_width, img_height)

    # Save the YOLO formatted annotations to the corresponding label file
    with open(lb_path, 'w') as f:
        f.write('\n'.join(yolo_annotations))


In [ ]:
# Loop through each pair of testing images and labels
for t_img, t_lb in tqdm(test):
    # Define the path for copying the image to the validation split directory
    im_path = valid_path / t_img.name

    # Define the path for copying the label to the validation split directory
    lb_path = valid_path / t_lb.name

    # Copy the testing image to the designated path
    shutil.copy(t_img, im_path)

    # Copy the testing label to the designated path
    shutil.copy(t_lb, lb_path)

    # Open the copied image to get its dimensions
    img = Image.open(im_path)
    img_width, img_height = img.size

    # Convert the KITTI format label to YOLO format
    yolo_annotations = convert_kitti_to_yolo(t_lb, img_width, img_height)

    # Save the YOLO formatted annotations to the corresponding label file
    with open(lb_path, 'w') as f:
        f.write('\n'.join(yolo_annotations))


In [ ]:
# Initialize a string for the YAML file content
# 'names:' indicates the start of the class names section
yaml_file = 'names:\n'

# Add each class name to the YAML file content, with a preceding hyphen and space
yaml_file += '\n'.join(f'- {c}' for c in classes)

# Add the number of classes ('nc') to the YAML file content
yaml_file += f'\nnc: {len(classes)}'

# Add the paths for the training and validation data to the YAML file content
yaml_file += f'\ntrain: {str(train_split_path)}\nval: {str(valid_path)}'

# Write the YAML file content to 'data.yaml'
with open('data.yaml', 'w') as f:
    f.write(yaml_file)

# Display the content of 'data.yaml' to verify its correctness
!cat data.yaml


In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import pathlib # Importing the pathlib module

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name,
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

  return model

model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
model = load_model(model_name)

In [ ]:
# Define early stopping criterion to prevent overfitting
early_stopping = EarlyStopping(
    monitor='accuracy',  # Monitor the 'accuracy' metric to decide when to stop training
    min_delta=0.01,  # Minimum change in the monitored metric to qualify as an improvement (0.01 means 1% improvement)
    patience=3,  # Number of epochs with no improvement after which training will be stopped (wait for 2 epochs with no improvement)
    restore_best_weights=True,  # Restore the model weights from the epoch with the best accuracy once training stops
)

In [ ]:

import tensorflow as tf

# Specify the use of a specific GPU device for training (e.g., GPU:1)
with tf.device('/GPU:1'):
    # Set the number of training epochs
    epochs = 1

    # Train the YOLOv8 model with the specified parameters
    train_results = model.train(
        data='data.yaml',       # Path to the data configuration file
        epochs=epochs,          # Number of training epochs
        patience=3,             # Early stopping patience; stop training if no improvement after 3 epochs
        mixup=0.5,              # Mixup augmentation rate
        project='yolov8_Assignment',  # Name of the project for saving results
    )


In [ ]:
# Validate the trained YOLOv8 model using the specified data configuration file
valid_results = model.val(data='data.yaml')


In [ ]:
# Get all metrics as a dictionary for comprehensive evaluation
all_metrics = valid_results.results_dict

# Print all metrics with descriptive labels
print("\nAll Metrics:")
for key, value in all_metrics.items():
    print(f"{key}: {value}")


In [ ]:
# Create a new figure with a specified size
plt.figure(figsize=(10, 20))

# Open and display the image of training results from the specified path
plt.imshow(Image.open('yolov8_Assignment/train/results.png'))

# Remove the axis for better visualization of the image
plt.axis('off')

# Display the image
plt.show()


In [ ]:
plt.figure(figsize=(10,20))
plt.imshow(Image.open('yolov8_Assignment/train2/confusion_matrix.png'))
plt.axis('off')
plt.show()

In [ ]:
colors = {
    'Car': (255, 0, 0),          # Red
    'Pedestrian': (0, 255, 0),   # Green
    'Van': (0, 0, 255),          # Blue
    'Cyclist': (255, 255, 0),    # Cyan
    'Truck': (255, 0, 255),      # Magenta
    'Misc': (0, 255, 255),       # Yellow
    'Tram': (128, 0, 128),       # Purple
    'Person_sitting': (0, 128, 0) # Dark Green
}

image_files = os.listdir(test_path)

# Randomly select 10 images (or fewer if there are not enough)
num_images_to_show = 10
image_paths = random.sample([os.path.join(test_path, img) for img in image_files], min(num_images_to_show, len(image_files)))

processed_images = []

# Get the list of class names from the 'classes' dictionary
class_names = list(classes.keys())
# Continue with the loop as before
for image_path in image_paths:
    # Run inference on the image
    results = model.predict(image_path)

    # Read the image once
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB for display

    # Accessing the predictions
    for result in results:
        boxes = result.boxes  # Bounding boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]  # Top-left and bottom-right corners
            confidence = box.conf[0]      # Confidence score
            class_id = int(box.cls[0])    # Class ID

            # Get the class name from the class ID
            class_name = class_names[class_id] if class_id < len(class_names) else "Unknown"

            # Print the class name and confidence value for all classes
            print(f"Class Name: {class_name}, Class ID: {class_id}, Confidence: {confidence}, BBox: [{x1}, {y1}, {x2}, {y2}]")

            # Create the label text with class name and confidence
            label_text = f"{class_name} ({confidence:.2f})"

            # Get the color for the current class
            color = colors.get(class_name, (255, 255, 255))  # Default to white if class not found

            # Draw the bounding box on the image with the specified color
            cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)

            # Add label text on top of the bounding box
            cv2.putText(img, label_text, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    # Append the processed image to the list
    processed_images.append(img)

    # Display the processed images in a grid
plt.figure(figsize=(20, 20))
for i, processed_img in enumerate(processed_images):
    plt.subplot(4, 3, i + 1)  # Adjust the grid size as needed
    plt.imshow(processed_img)
    plt.axis('off')  # Hide axis
plt.tight_layout()
plt.show()

In [ ]:
# Function to capture a single frame from the webcam
def capture_frame(quality=0.8):
    js = Javascript('''
        async function captureFrame(quality) {
            const video = document.createElement('video');
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            // Create video element and start stream
            video.srcObject = stream;
            await video.play();

            // Capture a single frame to a canvas
            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);

            // Stop the stream and return the image data
            stream.getTracks().forEach(track => track.stop());
            return canvas.toDataURL('image/jpeg', quality);
        }
    ''')
    display(js)
    data = eval_js(f'captureFrame({quality})')
    binary = b64decode(data.split(',')[1])
    img_array = np.frombuffer(binary, np.uint8)
    return cv2.imdecode(img_array, cv2.IMREAD_COLOR)

# Function to perform object detection on a single frame
def detect_objects_on_frame(frame, model, labels):
    # Perform inference using the YOLOv8 model
    results = model(frame)

    # Get detections and class names
    detections = results[0].boxes.data.cpu().numpy()
    class_ids = detections[:, -1].astype(int)

    for detection in detections:
        x1, y1, x2, y2, confidence, class_id = detection

        # Filter weak detections (adjust threshold as needed)
        if confidence > 0.3:

            # Draw bounding box and label
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            label = f"{list(labels.keys())[list(labels.values()).index(class_id)]}: {confidence * 100:.2f}%"  # Get label from class_id
            cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return frame

# Main function to run detection in Colab
def run_object_detection_colab(model, labels):
    print("Press 'Stop' in Colab to exit the loop.")
    try:
        while True:
            # Capture a frame
            frame = capture_frame()
            if frame is None:
                print("No frame captured.")
                break

            # Perform object detection
            processed_frame = detect_objects_on_frame(frame, model, labels)

            # Display the processed frame
            cv2_imshow(processed_frame)
    except KeyboardInterrupt:
        print("Detection stopped.")

# Run the object detection
run_object_detection_colab(model, classes)
